# Preprocessing and Feature Engineering

In this notebook I will create a clean dataset ready to model through feature engineering and standardization.

First load the data from EDA.  

In [1]:
import pandas as pd

In [3]:
train = pd.read_csv('train_w_aggregated_weather.csv',index_col=0)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4616 entries, 0 to 4615
Data columns (total 70 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    4616 non-null   object 
 1   Trap                    4616 non-null   object 
 2   Address                 4616 non-null   object 
 3   Block                   4616 non-null   int64  
 4   Street                  4616 non-null   object 
 5   AddressNumberAndStreet  4616 non-null   object 
 6   Latitude                4616 non-null   float64
 7   Longitude               4616 non-null   float64
 8   AddressAccuracy         4616 non-null   int64  
 9   RESTUANS                4616 non-null   int64  
 10  PIPIENS                 4616 non-null   int64  
 11  SALINARIUS              4616 non-null   int64  
 12  TERRITANS               4616 non-null   int64  
 13  TARSALIS                4616 non-null   int64  
 14  ERRATICUS               4616 non-null   

In [4]:
### drop mosquito species columns, won't have that for predicting 
train = train.drop(['RESTUANS','PIPIENS','SALINARIUS','TERRITANS','TARSALIS','ERRATICUS'],axis=1)

In [ ]:
### Drop extraneous columns regarding trap location
train = train.drop(['Address','Block','Street','AddressNumberAndStreet'])

In [5]:
train

,Date,Trap,Address,Block,Street,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,...,WindDirection_14days,Sunrise(hrs)_14days,Sunset(hrs)_14days,CodeSum_14days,RA_7days,VC_7days,HZ_7days,FG_7days,TS_7days,BR_7days
0,2007-05-29,T002,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,2,...,176.000000,4.428889,19.181111,RA BR RA BR RA VCTS RA HZ TSRA R...,5,0,3,0,1,3
1,2007-05-29,T007,"6200 North Mandell Avenue, Chicago, IL 60646, USA",62,N MANDELL AVE,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,...,176.000000,4.428889,19.181111,RA BR RA BR RA VCTS RA HZ TSRA R...,5,0,3,0,1,3
2,2007-05-29,T015,"7900 West Foster Avenue, Chicago, IL 60656, USA",79,W FOSTER AVE,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,5,...,176.000000,4.428889,19.181111,RA BR RA BR RA VCTS RA HZ TSRA R...,5,0,3,0,1,3
3,2007-05-29,T045,"1500 West Webster Avenue, Chicago, IL 60614, USA",15,W WEBSTER AVE,"1500 W WEBSTER AVE, Chicago, IL",41.921600,-87.666455,8,2,...,161.333333,4.428889,19.181111,TSRA RA BR RA BR TSRA RA RA HZ RA ...,5,0,2,0,1,4
4,2007-05-29,T046,"2500 West Grand Avenue, Chicago, IL 60654, USA",25,W GRAND AVE,"2500 W GRAND AVE, Chicago, IL",41.891118,-87.654491,8,1,...,161.333333,4.428889,19.181111,TSRA RA BR RA BR TSRA RA RA HZ RA ...,5,0,2,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4611,2013-09-26,T231,"5800 North Ridge Avenue, Chicago, IL 60660, USA",58,N RIDGE AVE,"5800 N RIDGE AVE, Chicago, IL",41.987280,-87.666066,8,5,...,136.666667,5.593333,17.917778,RA RA BR TS TSRA RA FG BR HZ ...,2,0,1,1,2,2
4612,2013-09-26,T232,"1700 North Ashland Avenue, Chicago, IL 60622, USA",17,N ASHLAND AVE,"1700 N ASHLAND AVE, Chicago, IL",41.912563,-87.668055,9,1,...,146.666667,5.593333,17.917778,RA DZ BR TS HZ RA BR HZ,1,0,1,0,0,1
4613,2013-09-26,T233,"7100 North Harlem Avenue, Chicago, IL 60631, USA",71,N HARLEM AVE,"7100 N HARLEM AVE, Chicago, IL",42.009876,-87.807277,9,5,...,136.666667,5.593333,17.917778,RA RA BR TS TSRA RA FG BR HZ ...,2,0,1,1,2,2
4614,2013-09-26,T235,"4200 West 65th Street, Chicago, IL 60621, USA",42,W 65TH ST,"4200 W 65TH ST, Chicago, IL",41.776428,-87.627096,8,1,...,146.666667,5.593333,17.917778,RA DZ BR TS HZ RA BR HZ,1,0,1,0,0,1
